In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
# Function to load and preprocess a video
def process_video(video_path, label, target_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize frame
        frame = cv2.resize(frame, target_size)

        # Normalize pixel values
        frame = frame / 255.0

        frames.append(frame)

    cap.release()
    return frames, [label] * len(frames)

In [3]:
import imgaug.augmenters as iaa

# Function to apply data augmentation to a list of frames
def augment_data(frames):
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),  # Horizontal flip with a 50% probability
        iaa.GaussianBlur(sigma=(0, 1.0)),  # Gaussian blur with a variable sigma
        iaa.Affine(rotate=(-10, 10)),  # Rotation in the range [-10, 10] degrees
        # Add more augmentation techniques as needed
    ])

    augmented_frames = seq(images=frames)
    return augmented_frames


In [5]:
# Function to load and preprocess videos from a folder
def process_videos_from_folder(folder_path, label, target_size=(224, 224)):
    data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(folder_path, filename)
            frames, labels = process_video(video_path, label, target_size)
            #frames = augment_data(frames)
            data.extend(zip(frames, labels))
            break
            

    return data

In [7]:
# Example usage
dataset_folder = "./dataset"

# Load and process drowsy videos
drowsy_data = process_videos_from_folder(os.path.join(dataset_folder, "drowsy"), label='drowsy')


In [8]:
# Load and process alert videos
alert_data = process_videos_from_folder(os.path.join(dataset_folder, "alert"), label='alert')


In [73]:
len(drowsy_data), len(alert_data)

(1516, 1520)

In [10]:
# Combine alert and drowsy data
all_data = alert_data + drowsy_data
np.random.shuffle(all_data)

# Separate features and labels
features, labels = zip(*all_data)


In [75]:
# Convert labels to numeric format
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [64]:
# Function to detect faces and eyes in a frame
def detect_face_and_eyes(frame):
        
    #frame = frame.astype('uint8')
    
    normalized_frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)
    frame_u8 = np.uint8(normalized_frame)
    
    gray = cv2.cvtColor(frame_u8, cv2.COLOR_BGR2GRAY)

    # cv2.imshow('Grayscale Image', gray)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    # print(gray.shape)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    #print(faces)

    left_eye_regions = []
    right_eye_regions = []

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x-10:x+w+10]
        eyes_left = eye_cascade_left.detectMultiScale(roi_gray)
        eyes_right = eye_cascade_right.detectMultiScale(roi_gray)
            
         # Use the first detected eye for each side
        if len(eyes_left) > 0:
            left_eye = roi_gray[eyes_left[0][1]:eyes_left[0][1]+eyes_left[0][3], eyes_left[0][0]:eyes_left[0][0]+eyes_left[0][2]]
            left_eye_regions.append(left_eye)
        else:
            left_eye_regions.append(None)

        if len(eyes_right) > 0:
            right_eye = roi_gray[eyes_right[0][1]:eyes_right[0][1]+eyes_right[0][3], eyes_right[0][0]:eyes_right[0][0]+eyes_right[0][2]]
            right_eye_regions.append(right_eye)
        else:
            right_eye_regions.append(None)

    return left_eye_regions, right_eye_regions

In [66]:
# Function to process frames and labels with face detection
def process_frames_with_eyes(frames, target_size=(224, 224)):
    left_eye_frames = []
    right_eye_frames = []

    for frame in frames:
    
        # Detect face and eyes
        left_eye_regions, right_eye_regions = detect_face_and_eyes(frame)

        for left_eye, right_eye in zip(left_eye_regions, right_eye_regions):
            # Resize left eye region

            if (left_eye is None):
                left_eye_frames.append(None)
                right_eye_frames.append(None)
                continue
                

            if (right_eye is None):
                right_eye_frames.append(None)
                left_eye_frames.append(None)
                continue

            left_eye = cv2.resize(left_eye, target_size)

            # Resize right eye region
            right_eye = cv2.resize(right_eye, target_size)

            # Normalize pixel values
            left_eye = left_eye / 255.0
            right_eye = right_eye / 255.0

            left_eye_frames.append(left_eye)
            right_eye_frames.append(right_eye)
        
        

    return left_eye_frames, right_eye_frames


In [67]:
# Process frames with face detection
left_eye_frames, right_eye_frames = process_frames_with_eyes(features)

In [68]:
len(left_eye_frames), len(right_eye_frames)

(536, 536)

In [69]:
# Assume X_train_left, X_train_right, and labels are lists or arrays containing features and labels
# detected using detect_face_and_eyes function

left_eye_filtered = []
right_eye_filtered = []
labels_filtered = []

for frame_index in range(len(left_eye_frames)):
    left_eye_features = left_eye_frames[frame_index]
    right_eye_features = right_eye_frames[frame_index]
    label = labels[frame_index]

    # Check if either left or right eye features are not None
    if left_eye_features is not None and right_eye_features is not None:
            left_eye_filtered.append(left_eye_features)
            right_eye_filtered.append(right_eye_features)
            labels_filtered.append(label)


print(len(left_eye_filtered), len(right_eye_filtered), len(labels_filtered))


175 175 175


In [72]:
# Assuming left_eye_frames and right_eye_frames are lists of left and right eye frames
left_eye_frames = np.array(left_eye_filtered)
right_eye_frames = np.array(right_eye_filtered)
labels_filtered = np.array(labels_filtered)

# Convert labels to numeric format
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels_filtered)
labels_encoded = np.array(labels_encoded)


In [73]:
print(left_eye_frames.shape, right_eye_frames.shape, labels_encoded.shape)

(175, 224, 224) (175, 224, 224) (175,)


In [71]:

# Concatenate left and right eye frames along the last axis
combined_eye_frames = np.concatenate((left_eye_frames, right_eye_frames), axis=-1)
len(combined_eye_frames), len(labels_encoded)



In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train_both_eyes, X_test_both_eyes, y_train, y_test = train_test_split(combined_eye_frames, labels_encoded, test_size=0.2, random_state=42)

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)